In [1]:
import tensorflow as tf
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import os
import pathlib
dataset_path = '/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset'
data_dir = pathlib.Path(dataset_path)
data_dir

PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset')

In [3]:
!pip install pillow

In [4]:
list(data_dir.glob('*/*.png'))[:5]

[PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Monkeypox/monkeypox235.png'),
 PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Monkeypox/monkeypox273.png'),
 PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Monkeypox/monkeypox270.png'),
 PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Monkeypox/monkeypox132.png'),
 PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Monkeypox/monkeypox258.png')]

In [5]:
chickenpox = list(data_dir.glob('Chickenpox/*'))
chickenpox[:5]

[PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Chickenpox/chickenpox53.png'),
 PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Chickenpox/chickenpox56.png'),
 PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Chickenpox/chickenpox40.png'),
 PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Chickenpox/chickenpox96.png'),
 PosixPath('/kaggle/input/monkeypoxskinimagedataset/Monkeypox Skin Image Dataset/Chickenpox/chickenpox59.png')]

In [6]:
disease_img_dict = {
    'chickenpox': list(data_dir.glob('Chickenpox/*')),
    'measles': list(data_dir.glob('Measles/*')),
    'monkeypox': list(data_dir.glob('Monkeypox/*')),
    'normal': list(data_dir.glob('Normal/*'))
}

In [7]:
disease_labels_dict = {
    'chickenpox': 0,
    'measles': 1,
    'monkeypox': 2,
    'normal': 3
}

In [8]:
!pip install opencv-python

In [9]:
import cv2
img = cv2.imread(str(disease_img_dict['measles'][0]))
img.shape

(224, 224, 3)

In [10]:
X, y = [], []
for disease_name, images in disease_img_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        X.append(img)
        y.append(disease_labels_dict[disease_name])

In [11]:
X = np.array(X)
y = np.array(y)

In [12]:
class_labels = {'chickenpox': 0, 'measles': 0, 'monkeypox': 0, 'normal': 0}
for i in y:
    if i in class_labels:
        class_labels[i]+=1
for class_label, counts in class_labels.items():
    print(f"{class_label}: {counts}")

chickenpox: 0
measles: 0
monkeypox: 0
normal: 0


In [13]:
X_2d = X.reshape((X.shape[0], -1))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_2d, y, test_size = 0.2)

In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy = 'minority')
X_sm, y_sm = smote.fit_resample(X_train, y_train)

In [15]:
from collections import Counter
min_class_size = min(Counter(y_sm).values())
undersampled_indices = []
for class_label in set(y_sm):
    class_indices = np.where(y_sm == class_label)[0]
    undersampled_indices.extend(np.random.choice(class_indices, min_class_size, replace=False))
X_train_undersampled = X_sm[undersampled_indices]
y_train_undersampled = y_sm[undersampled_indices]
print("Class Counts After undersampling:")
for class_label in set(y_train_undersampled):
    print(f"{class_label}: {np.sum(y_train_undersampled == class_label)} samples")

Class Counts After undersampling:
0: 85 samples
1: 85 samples
2: 85 samples
3: 85 samples


In [16]:
from sklearn.utils import shuffle
X_train_undersampled, y_train_undersampled = shuffle(X_train_undersampled, y_train_undersampled, random_state = 42)
X_train_undersampled = X_train_undersampled.reshape((-1, 224, 224, 3))
X_train_undersampled.shape

(340, 224, 224, 3)

In [17]:
data_augmentation = ([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape = (224, 224, 3)),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1),
    layers.experimental.preprocessing.RandomTranslation(height_factor = 0.1, width_factor = 0.1)
])

In [18]:
X_train_undersampled = X_train_undersampled/255
X_test_scaled = X_test/255

In [26]:
import tensorflow_hub as hub
feature_extractor_model = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [27]:
model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(4)
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 2048)              23564800  
                                                                 
 dense_1 (Dense)             (None, 4)                 8196      
                                                                 
Total params: 23572996 (89.92 MB)
Trainable params: 8196 (32.02 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [28]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_undersampled, y_train_undersampled, epochs=25)

Epoch 1/25
11/11 [==============================] - 5s 85ms/step - loss: 1.3019 - acc: 0.4941
Epoch 2/25
11/11 [==============================] - 1s 79ms/step - loss: 0.6920 - acc: 0.7529
Epoch 3/25
11/11 [==============================] - 1s 79ms/step - loss: 0.4818 - acc: 0.8265
Epoch 4/25
11/11 [==============================] - 1s 79ms/step - loss: 0.3629 - acc: 0.8912
Epoch 5/25
11/11 [==============================] - 1s 78ms/step - loss: 0.2969 - acc: 0.9118
Epoch 6/25
11/11 [==============================] - 1s 80ms/step - loss: 0.2421 - acc: 0.9265
Epoch 7/25
11/11 [==============================] - 1s 79ms/step - loss: 0.2019 - acc: 0.9529
Epoch 8/25
11/11 [==============================] - 1s 78ms/step - loss: 0.1722 - acc: 0.9647
Epoch 9/25
11/11 [==============================] - 1s 80ms/step - loss: 0.1501 - acc: 0.9706
Epoch 10/25
11/11 [==============================] - 1s 79ms/step - loss: 0.1305 - acc: 0.9735
Epoch 11/25
11/11 [==============================] - 1s 79m

In [29]:
X_test_reshaped = X_test_scaled.reshape(-1, 224, 224, 3)
model.evaluate(X_test_reshaped, y_test)

5/5 [==============================] - 1s 89ms/step - loss: 0.5259 - acc: 0.7922


[0.5259263515472412, 0.7922077775001526]

In [30]:
from sklearn.metrics import confusion_matrix, classification_report
y_preds = model.predict(X_test_reshaped)
y_preds = np.round(y_preds)
y_true_classes = y_test
y_pred_classes = np.argmax(y_preds, axis=1)
print(classification_report(y_true_classes, y_pred_classes))

5/5 [==============================] - 1s 90ms/step
              precision    recall  f1-score   support

           0       0.53      0.82      0.64        22
           1       0.47      0.67      0.55        12
           2       0.95      0.67      0.79        61
           3       0.90      0.92      0.91        59

    accuracy                           0.79       154
   macro avg       0.71      0.77      0.72       154
weighted avg       0.83      0.79      0.79       154



In [31]:
import pickle
import os
file_path = "diseases.pkl"
with open(file_path, "wb") as f:
    pickle.dump(model, f)